In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
%pwd

'e:\\Machine Learning\\MLFlow_Project_by_Krish_Naik\\Kidney-Disease-Classification-Deep-Learning-Project\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'e:\\Machine Learning\\MLFlow_Project_by_Krish_Naik\\Kidney-Disease-Classification-Deep-Learning-Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
  root_dir: Path
  path_of_model: Path
  training_data: Path
  all_params: dict
  mlflow_uri: str
  params_image_size: list
  params_batch_size: int

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        config = self.config.evaluation
        create_directories([config.root_dir])

        eval_config = EvaluationConfig(
            root_dir = Path(config.root_dir),
            path_of_model = Path(config.model_path),
            training_data = Path(config.training_data),
            all_params = self.params,
            mlflow_uri = config.mlflow_uri,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        
        return eval_config

In [9]:
import tensorflow as tf
print(tf.__version__)
from pathlib import Path
import mlflow
import mlflow.keras
print(mlflow.__version__)
from urllib.parse import urlparse
import dagshub
print(dagshub.__version__)

2.20.0
2.16.0
0.3.28


In [10]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path)-> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"Loss": self.score[0], "Accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        # Modern DagsHub Integration
        dagshub.init(
            repo_owner='vipul100699', 
            repo_name='Kidney-Disease-Classification-Deep-Learning-Project', 
            mlflow=True
        )
        
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(
                    self.model,
                    "model",
                    registered_model_name="VGG16Model"
                )
            else:
                mlflow.keras.log_model(self.model, "model")

In [11]:
# Building the evaluation pipeline
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-09-04 01:17:17,683: INFO: common: yaml file: <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='cp1252'> loaded successfully]
[2025-09-04 01:17:17,689: INFO: common: yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='cp1252'> loaded successfully]
[2025-09-04 01:17:17,694: INFO: common: Created directory at: artifacts]
[2025-09-04 01:17:17,696: INFO: common: Created directory at: artifacts/model_evaluation]
[2025-09-04 01:17:20,812: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 139 images belonging to 2 classes.


e:\Machine Learning\MLFlow_Project_by_Krish_Naik\Kidney-Disease-Classification-Deep-Learning-Project\kidney2\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 118s 12s/step - accuracy: 0.4820 - loss: 15.3254
[2025-09-04 01:19:20,838: INFO: common: JSON file saved at: scores.json]


Initialized MLflow to track repo "vipul100699/Kidney-Disease-Classification-Deep-Learning-Project"

[2025-09-04 01:19:25,292: INFO: helpers: Initialized MLflow to track repo "vipul100699/Kidney-Disease-Classification-Deep-Learning-Project"]


Repository vipul100699/Kidney-Disease-Classification-Deep-Learning-Project initialized!

[2025-09-04 01:19:25,322: INFO: helpers: Repository vipul100699/Kidney-Disease-Classification-Deep-Learning-Project initialized!]


2025/09/04 01:19:31 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/09/04 01:22:10 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\VIPULG~1\AppData\Local\Temp\tmpcddfj023\model, flavor: keras). Fall back to return ['keras==3.11.3']. Set logging level to DEBUG to see the full traceback. 
2025/09/04 01:22:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'VGG16Model'.
2025/09/04 01:23:15 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
2025/09/04 01:23:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run serious-crab-753 at: https://dagshub.com/vipul100699/Kidney-Dis